In [1]:
import torchvision as tv
import phototour
import torch
from tqdm import tqdm
import numpy as np
import torch.nn as nn
import math

import tfeat_model
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

# data import

In [2]:
lib_train = phototour.PhotoTour('.','liberty', download=True, train=True, mode = 'Quadruwithglobal', augment = True, nsamples=409600)
yos_train = phototour.PhotoTour('.','yosemite', download=True, train=True, mode = 'Quadruwithglobal', augment = True)
nd_train = phototour.PhotoTour('.','notredame', download=True, train=True, mode = 'Quadruwithglobal', augment = True)

# Found cached data ./liberty.pt
# Found cached data ./yosemite.pt
# Found cached data ./notredame.pt


# train

## 1. train : notredam & test : yosemite

In [4]:
eval_db = phototour.PhotoTour('.','yosemite', download=True, train=False)

train_db = nd_train
train_name = 'notredame'
eval_name = 'yosemite'

tfeat = tfeat_model.TNet()
tfeat = tfeat.cuda()

# this kind of works
optimizer = optim.Adam(tfeat.parameters(), lr=1e-3, betas=(0., 0.999))
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.8)

seed=42
torch.manual_seed(seed)
np.random.seed(seed)
# cv2.setRNGSeed(seed)

train_loader = torch.utils.data.DataLoader(train_db,
                                             batch_size=300, shuffle=False,
                                             num_workers=30)

eval_loader = torch.utils.data.DataLoader(eval_db,batch_size=1024, shuffle=False,num_workers=32)

fpr_per_epoch = []

for e in range(100):
    tfeat.train()
    for batch_idx, (data_a, data_p, data_n1, data_n2) in tqdm(enumerate(train_loader)):
        data_a = data_a.unsqueeze(1).float().cuda()
        data_p = data_p.unsqueeze(1).float().cuda()
        data_n1 = data_n1.unsqueeze(1).float().cuda()
        data_n2 = data_n2.unsqueeze(1).float().cuda()
        out_a, out_p, out_n1,out_n2 = tfeat(data_a), tfeat(data_p), tfeat(data_n1), tfeat(data_n2)

        hyperparameter = tfeat_model.newQuadruplet(margin1=2.0, margin2=1.0,gamma=0.8,ramda=0.8,t1=0.1,t2=0.05)
        loss=hyperparameter.loss(out_a,out_p,out_n1,out_n2,swap=False)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

#     print(loss)
    tfeat.eval()
    l = np.empty((0,))
    d = np.empty((0,))
    # evaluate the network after each epoch
    for batch_idx, (data_l, data_r, lbls) in enumerate(eval_loader):
        data_l = data_l.unsqueeze(1).float().cuda()
        data_r = data_r.unsqueeze(1).float().cuda()
        out_l, out_r = tfeat(data_l), tfeat(data_r)
        dists = torch.norm(out_l - out_r, 2, 1).detach().cpu().numpy()
        l = np.hstack((l, lbls.numpy()))
        d = np.hstack((d, dists))

    # FPR95 code from Yurun Tian
    d = torch.from_numpy(d)
    l = torch.from_numpy(l)
    dist_pos = d[l == 1]
    dist_neg = d[l != 1]
    dist_pos, indice = torch.sort(dist_pos)
    loc_thr = int(np.ceil(dist_pos.numel() * 0.95))
    thr = dist_pos[loc_thr]
    fpr95 = float(dist_neg.le(thr).sum()) / dist_neg.numel()
    print(e, fpr95)
    fpr_per_epoch.append([e, fpr95])
    scheduler.step()
    np.savetxt(train_name+ eval_name+'_quadru_fpr_without_swap.txt', np.array(fpr_per_epoch), delimiter=',')
    #torch.save(tfeat.state_dict(), train_name + '-quadru.params')  

# Found cached data ./yosemite.pt


0it [00:00, ?it/s]Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f13b62cfe60>Exception ignored in: 
<function _MultiProcessingDataLoaderIter.__del__ at 0x7f13b62cfe60>Exception ignored in: 
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/cvlab/anaconda3/envs/pytorch/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1324, in __del__
<function _MultiProcessingDataLoaderIter.__del__ at 0x7f13b62cfe60>Exception ignored in:   File "/home/cvlab/anaconda3/envs/pytorch/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1324, in __del__

    <function _MultiProcessingDataLoaderIter.__del__ at 0x7f13b62cfe60>self._shutdown_workers()Traceback (most recent call last):


      File "/home/cvlab/anaconda3/envs/pytorch/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1324, in __del__
Exception ignored in: self._shutdown_workers()Exception ignored in: Exception ignored in:   File "/home/cv

  File "/home/cvlab/anaconda3/envs/pytorch/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1316, in _shutdown_workers
self._shutdown_workers()  File "/home/cvlab/anaconda3/envs/pytorch/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1316, in _shutdown_workers
    

    Traceback (most recent call last):
self._shutdown_workers()      File "/home/cvlab/anaconda3/envs/pytorch/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1324, in __del__
  File "/home/cvlab/anaconda3/envs/pytorch/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1316, in _shutdown_workers


  File "/home/cvlab/anaconda3/envs/pytorch/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1316, in _shutdown_workers

assert self._parent_pid == os.getpid(), 'can only test a child process'
      File "/home/cvlab/anaconda3/envs/pytorch/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1316, in _shutdown_workers
Traceback (most re

: can only test a child process: assert self._parent_pid == os.getpid(), 'can only test a child process'AssertionError

: can only test a child processself._shutdown_workers(): 
assert self._parent_pid == os.getpid(), 'can only test a child process'can only test a child process  File "/home/cvlab/anaconda3/envs/pytorch/lib/python3.7/multiprocessing/process.py", line 151, in is_alive
self._shutdown_workers()  File "/home/cvlab/anaconda3/envs/pytorch/lib/python3.7/multiprocessing/process.py", line 151, in is_alive
    
self._shutdown_workers(): 

  File "/home/cvlab/anaconda3/envs/pytorch/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1324, in __del__

  File "/home/cvlab/anaconda3/envs/pytorch/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1316, in _shutdown_workers
can only test a child process
AssertionErrorcan only test a child process    can only test a child process
    : assert self._parent_pid == os.getpid(), 'can only test a child process'



0 0.16486


...

97 0.06484


3334it [02:06, 26.31it/s]


98 0.06758


3334it [02:08, 26.01it/s]


99 0.06694


## 2. train : notredam & test : liberty

In [5]:
eval_db = phototour.PhotoTour('.','liberty', download=True, train=False)

train_db = nd_train
train_name = 'notredame'
eval_name = 'liberty'

tfeat = tfeat_model.TNet()
tfeat = tfeat.cuda()

# this kind of works
optimizer = optim.Adam(tfeat.parameters(), lr=1e-3, betas=(0., 0.999))
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.8)

seed=42
torch.manual_seed(seed)
np.random.seed(seed)
# cv2.setRNGSeed(seed)

train_loader = torch.utils.data.DataLoader(train_db,
                                             batch_size=300, shuffle=False,
                                             num_workers=30)

eval_loader = torch.utils.data.DataLoader(eval_db,batch_size=1024, shuffle=False,num_workers=32)

fpr_per_epoch = []

for e in range(100):
    tfeat.train()
    for batch_idx, (data_a, data_p, data_n1, data_n2) in tqdm(enumerate(train_loader)):
        data_a = data_a.unsqueeze(1).float().cuda()
        data_p = data_p.unsqueeze(1).float().cuda()
        data_n1 = data_n1.unsqueeze(1).float().cuda()
        data_n2 = data_n2.unsqueeze(1).float().cuda()
        out_a, out_p, out_n1,out_n2 = tfeat(data_a), tfeat(data_p), tfeat(data_n1), tfeat(data_n2)

        hyperparameter = tfeat_model.newQuadruplet(margin1=2.0, margin2=1.0,gamma=0.8,ramda=0.8,t1=0.4,t2=0.2)
        loss=hyperparameter.loss(out_a,out_p,out_n1,out_n2,swap=False)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

#     print(loss)
    tfeat.eval()
    l = np.empty((0,))
    d = np.empty((0,))
    # evaluate the network after each epoch
    for batch_idx, (data_l, data_r, lbls) in enumerate(eval_loader):
        data_l = data_l.unsqueeze(1).float().cuda()
        data_r = data_r.unsqueeze(1).float().cuda()
        out_l, out_r = tfeat(data_l), tfeat(data_r)
        dists = torch.norm(out_l - out_r, 2, 1).detach().cpu().numpy()
        l = np.hstack((l, lbls.numpy()))
        d = np.hstack((d, dists))

    # FPR95 code from Yurun Tian
    d = torch.from_numpy(d)
    l = torch.from_numpy(l)
    dist_pos = d[l == 1]
    dist_neg = d[l != 1]
    dist_pos, indice = torch.sort(dist_pos)
    loc_thr = int(np.ceil(dist_pos.numel() * 0.95))
    thr = dist_pos[loc_thr]
    fpr95 = float(dist_neg.le(thr).sum()) / dist_neg.numel()
    print(e, fpr95)
    fpr_per_epoch.append([e, fpr95])
    scheduler.step()
    np.savetxt(train_name+ eval_name+'_quadru_fpr_without_swap.txt', np.array(fpr_per_epoch), delimiter=',')
    #torch.save(tfeat.state_dict(), train_name+'-'+eval_name + '-quadru.params')  

# Found cached data ./liberty.pt


3334it [02:07, 26.17it/s]


0 0.2459


3334it [02:06, 26.39it/s]


1 0.19606


...

97 0.06854


3334it [02:07, 26.05it/s]


98 0.0677


3334it [02:06, 26.26it/s]


99 0.06712


## 3. train : yosemite & test : notredam

In [6]:
eval_db = phototour.PhotoTour('.','notredame', download=True, train=False)

train_db = yos_train
eval_name = 'notredame'
train_name = 'yosemite'

tfeat = tfeat_model.TNet()
tfeat = tfeat.cuda()

# this kind of works
optimizer = optim.Adam(tfeat.parameters(), lr=1e-3, betas=(0., 0.999))
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.8)

seed=42
torch.manual_seed(seed)
np.random.seed(seed)
# cv2.setRNGSeed(seed)

train_loader = torch.utils.data.DataLoader(train_db,
                                             batch_size=300, shuffle=False,
                                             num_workers=30)

eval_loader = torch.utils.data.DataLoader(eval_db,
                                             batch_size=1024, shuffle=False,

                                             num_workers=32)

fpr_per_epoch = []

for e in range(100):
    tfeat.train()
    for batch_idx, (data_a, data_p, data_n1, data_n2) in tqdm(enumerate(train_loader)):
        data_a = data_a.unsqueeze(1).float().cuda()
        data_p = data_p.unsqueeze(1).float().cuda()
        data_n1 = data_n1.unsqueeze(1).float().cuda()
        data_n2 = data_n2.unsqueeze(1).float().cuda()
        out_a, out_p, out_n1,out_n2 = tfeat(data_a), tfeat(data_p), tfeat(data_n1), tfeat(data_n2)

        hyperparameter = tfeat_model.newQuadruplet(margin1=2.0, margin2=1.0,gamma=0.8,ramda=0.8,t1=0.1)
        loss=hyperparameter.loss(out_a,out_p,out_n1,out_n2,swap=False)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


    tfeat.eval()
    l = np.empty((0,))
    d = np.empty((0,))
    # evaluate the network after each epoch
    for batch_idx, (data_l, data_r, lbls) in enumerate(eval_loader):
        data_l = data_l.unsqueeze(1).float().cuda()
        data_r = data_r.unsqueeze(1).float().cuda()
        out_l, out_r = tfeat(data_l), tfeat(data_r)
        dists = torch.norm(out_l - out_r, 2, 1).detach().cpu().numpy()
        l = np.hstack((l, lbls.numpy()))
        d = np.hstack((d, dists))

    # FPR95 code from Yurun Tian
    d = torch.from_numpy(d)
    l = torch.from_numpy(l)
    dist_pos = d[l == 1]
    dist_neg = d[l != 1]
    dist_pos, indice = torch.sort(dist_pos)
    loc_thr = int(np.ceil(dist_pos.numel() * 0.95))
    thr = dist_pos[loc_thr]
    fpr95 = float(dist_neg.le(thr).sum()) / dist_neg.numel()
    print(e, fpr95)
    fpr_per_epoch.append([e, fpr95])
    scheduler.step()
    np.savetxt(train_name+ eval_name+'_quadru_fpr_without_swap.txt', np.array(fpr_per_epoch), delimiter=',')
    #torch.save(tfeat.state_dict(), train_name + '-quadru.params')                    

# Found cached data ./notredame.pt


3334it [02:10, 25.48it/s]


0 0.15854


3334it [02:08, 25.94it/s]


1 0.10756


...

97 0.10096


3334it [02:07, 26.12it/s]


98 0.1006


3334it [02:06, 26.32it/s]


99 0.1039


## 4. train : yosemite & test : liberty

In [7]:
eval_db = phototour.PhotoTour('.','liberty', download=True, train=False)

train_db = yos_train
eval_name = 'notredame'
train_name = 'liberty'

tfeat = tfeat_model.TNet()
tfeat = tfeat.cuda()

# this kind of works
optimizer = optim.Adam(tfeat.parameters(), lr=1e-3, betas=(0., 0.999))
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.8)

seed=42
torch.manual_seed(seed)
np.random.seed(seed)
# cv2.setRNGSeed(seed)

train_loader = torch.utils.data.DataLoader(train_db,
                                             batch_size=300, shuffle=False,
                                             num_workers=30)

eval_loader = torch.utils.data.DataLoader(eval_db,
                                             batch_size=1024, shuffle=False,

                                             num_workers=32)

fpr_per_epoch = []

for e in range(100):
    tfeat.train()
    for batch_idx, (data_a, data_p, data_n1, data_n2) in tqdm(enumerate(train_loader)):
        data_a = data_a.unsqueeze(1).float().cuda()
        data_p = data_p.unsqueeze(1).float().cuda()
        data_n1 = data_n1.unsqueeze(1).float().cuda()
        data_n2 = data_n2.unsqueeze(1).float().cuda()
        out_a, out_p, out_n1,out_n2 = tfeat(data_a), tfeat(data_p), tfeat(data_n1), tfeat(data_n2)

        hyperparameter = tfeat_model.newQuadruplet(margin1=2.0, margin2=1.0,gamma=0.8,ramda=0.8,t1=0.4,t2=0.2)
        loss=hyperparameter.loss(out_a,out_p,out_n1,out_n2,swap=False)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


    tfeat.eval()
    l = np.empty((0,))
    d = np.empty((0,))
    # evaluate the network after each epoch
    for batch_idx, (data_l, data_r, lbls) in enumerate(eval_loader):
        data_l = data_l.unsqueeze(1).float().cuda()
        data_r = data_r.unsqueeze(1).float().cuda()
        out_l, out_r = tfeat(data_l), tfeat(data_r)
        dists = torch.norm(out_l - out_r, 2, 1).detach().cpu().numpy()
        l = np.hstack((l, lbls.numpy()))
        d = np.hstack((d, dists))

    # FPR95 code from Yurun Tian
    d = torch.from_numpy(d)
    l = torch.from_numpy(l)
    dist_pos = d[l == 1]
    dist_neg = d[l != 1]
    dist_pos, indice = torch.sort(dist_pos)
    loc_thr = int(np.ceil(dist_pos.numel() * 0.95))
    thr = dist_pos[loc_thr]
    fpr95 = float(dist_neg.le(thr).sum()) / dist_neg.numel()
    print(e, fpr95)
    fpr_per_epoch.append([e, fpr95])
    scheduler.step()
    np.savetxt(train_name+'-'+ eval_name+'_quadru_fpr_without_swap.txt', np.array(fpr_per_epoch), delimiter=',')
    #torch.save(tfeat.state_dict(), train_name+'-'+eval_name + '-quadru.params')                    

# Found cached data ./liberty.pt


3334it [02:07, 26.11it/s]


0 0.32456


3334it [02:07, 26.10it/s]


1 0.22354


...

97 0.19084


3334it [02:06, 26.26it/s]


98 0.18662


3334it [02:06, 26.33it/s]


99 0.19282


## 5. train : liberty & test : yosemite

In [8]:
eval_db = phototour.PhotoTour('.','yosemite', download=True, train=False)

train_db = lib_train
eval_name = 'yosemite'
train_name = 'liberty'

tfeat = tfeat_model.TNet()
tfeat = tfeat.cuda()

# this kind of works
optimizer = optim.Adam(tfeat.parameters(), lr=1e-3, betas=(0., 0.999))
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.8)

seed=42
torch.manual_seed(seed)
np.random.seed(seed)
# cv2.setRNGSeed(seed)

train_loader = torch.utils.data.DataLoader(train_db,
                                             batch_size=300, shuffle=False,
                                             num_workers=30)

eval_loader = torch.utils.data.DataLoader(eval_db,
                                             batch_size=1024, shuffle=False,

                                             num_workers=32)

fpr_per_epoch = []

for e in range(100):
    tfeat.train()
    for batch_idx, (data_a, data_p, data_n1, data_n2) in tqdm(enumerate(train_loader)):
        data_a = data_a.unsqueeze(1).float().cuda()
        data_p = data_p.unsqueeze(1).float().cuda()
        data_n1 = data_n1.unsqueeze(1).float().cuda()
        data_n2 = data_n2.unsqueeze(1).float().cuda()
        out_a, out_p, out_n1,out_n2 = tfeat(data_a), tfeat(data_p), tfeat(data_n1), tfeat(data_n2)

        hyperparameter = tfeat_model.newQuadruplet(margin1=2.0, margin2=1.0,gamma=0.8,ramda=0.8,t1=0.1)
        loss=hyperparameter.loss(out_a,out_p,out_n1,out_n2,swap=False)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    tfeat.eval()
    l = np.empty((0,))
    d = np.empty((0,))
    # evaluate the network after each epoch
    for batch_idx, (data_l, data_r, lbls) in enumerate(eval_loader):
        data_l = data_l.unsqueeze(1).float().cuda()
        data_r = data_r.unsqueeze(1).float().cuda()
        out_l, out_r = tfeat(data_l), tfeat(data_r)
        dists = torch.norm(out_l - out_r, 2, 1).detach().cpu().numpy()
        l = np.hstack((l, lbls.numpy()))
        d = np.hstack((d, dists))

    # FPR95 code from Yurun Tian
    d = torch.from_numpy(d)
    l = torch.from_numpy(l)
    dist_pos = d[l == 1]
    dist_neg = d[l != 1]
    dist_pos, indice = torch.sort(dist_pos)
    loc_thr = int(np.ceil(dist_pos.numel() * 0.95))
    thr = dist_pos[loc_thr]
    fpr95 = float(dist_neg.le(thr).sum()) / dist_neg.numel()
    print(e, fpr95)
    fpr_per_epoch.append([e, fpr95])
    scheduler.step()
    np.savetxt(train_name+ eval_name+'_quadru_fpr_without_swap.txt', np.array(fpr_per_epoch), delimiter=',')
    #torch.save(tfeat.state_dict(), train_name + '-quadru.params')                    

# Found cached data ./yosemite.pt


1366it [00:52, 26.26it/s]


0 0.23424


1366it [00:52, 26.19it/s]


1 0.21326


...

97 0.0788


1366it [00:53, 25.43it/s]


98 0.07886


1366it [00:53, 25.69it/s]


99 0.07974


## 6. train : liberty & test : notredame

In [9]:
eval_db = phototour.PhotoTour('.','notredame', download=True, train=False)

train_db = lib_train
eval_name = 'notredame'
train_name = 'liberty'

tfeat = tfeat_model.TNet()
tfeat = tfeat.cuda()

# this kind of works
optimizer = optim.Adam(tfeat.parameters(), lr=1e-3, betas=(0., 0.999))
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.8)

seed=42
torch.manual_seed(seed)
np.random.seed(seed)
# cv2.setRNGSeed(seed)

train_loader = torch.utils.data.DataLoader(train_db,
                                             batch_size=300, shuffle=False,
                                             num_workers=30)

eval_loader = torch.utils.data.DataLoader(eval_db,
                                             batch_size=1024, shuffle=False,

                                             num_workers=32)

fpr_per_epoch = []

for e in range(100):
    tfeat.train()
    for batch_idx, (data_a, data_p, data_n1, data_n2) in tqdm(enumerate(train_loader)):
        data_a = data_a.unsqueeze(1).float().cuda()
        data_p = data_p.unsqueeze(1).float().cuda()
        data_n1 = data_n1.unsqueeze(1).float().cuda()
        data_n2 = data_n2.unsqueeze(1).float().cuda()
        out_a, out_p, out_n1,out_n2 = tfeat(data_a), tfeat(data_p), tfeat(data_n1), tfeat(data_n2)

        hyperparameter = tfeat_model.newQuadruplet(margin1=2.0, margin2=1.0,gamma=0.8,ramda=0.8,t1=0.4,t2=0.2)
        loss=hyperparameter.loss(out_a,out_p,out_n1,out_n2,swap=False)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    tfeat.eval()
    l = np.empty((0,))
    d = np.empty((0,))
    # evaluate the network after each epoch
    for batch_idx, (data_l, data_r, lbls) in enumerate(eval_loader):
        data_l = data_l.unsqueeze(1).float().cuda()
        data_r = data_r.unsqueeze(1).float().cuda()
        out_l, out_r = tfeat(data_l), tfeat(data_r)
        dists = torch.norm(out_l - out_r, 2, 1).detach().cpu().numpy()
        l = np.hstack((l, lbls.numpy()))
        d = np.hstack((d, dists))

    # FPR95 code from Yurun Tian
    d = torch.from_numpy(d)
    l = torch.from_numpy(l)
    dist_pos = d[l == 1]
    dist_neg = d[l != 1]
    dist_pos, indice = torch.sort(dist_pos)
    loc_thr = int(np.ceil(dist_pos.numel() * 0.95))
    thr = dist_pos[loc_thr]
    fpr95 = float(dist_neg.le(thr).sum()) / dist_neg.numel()
    print(e, fpr95)
    fpr_per_epoch.append([e, fpr95])
    scheduler.step()
    np.savetxt(train_name+'-'+ eval_name+'_quadru_fpr_without_swap.txt', np.array(fpr_per_epoch), delimiter=',')
    #torch.save(tfeat.state_dict(), train_name+'-'+eval_name + '-quadru.params')                    

# Found cached data ./notredame.pt


1366it [00:52, 25.85it/s]


0 0.17994


1366it [00:53, 25.65it/s]


1 0.1507


...

97 0.02954


1366it [00:53, 25.72it/s]


98 0.03008


1366it [00:53, 25.49it/s]


99 0.03074
